In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

## KPI
Se han definido los siguientes KPI:

**KPI 1: Incrementar en un 5% trimestral la cantidad de reseñas del establecimiento gastronómico, tomando como base el total acumulado de reseñas desde el primer registro.** 

**Definición:** Este KPI mide la popularidad y visibilidad de los establecimientos en función del crecimiento en el número de reseñas obtenidas en un período determinado. Se enfoca en monitorear cómo las estrategias de marketing y servicio al cliente están incentivando a los consumidores a dejar reseñas.  
Para establecer una línea de referencia inicial, los datos del trimestre anterior se calcularán utilizando los promedios de las reseñas obtenidas por los locales de la competencia directa en el mismo período. Esto permitirá evaluar el rendimiento del establecimiento en relación con sus competidores más cercanos.

**Fórmula:**

*KPI = ((Total Acumulado Reseñas (tri actual) − Total Acumulado Reseñas (tri inicial)) / Total Acumulado Reseñas (tri inicial)) x 100%*

*Establecer objetivo:*
*Objetivo KPI = Total Acumulado Reseñas (tri inicial) × (1 + 0.05) ^ Número de Trimestres Transcurridos*

 --

**Aclaración** El punto de partida fue: 
"Incrementar en un 5% trimestral la cantidad de reseñas del establecimiento gastronómico". <br>
Fórmula inicial: *KPI = ((Cant. Reseñas (tri actual) − Cant. Reseñas (tri ant)) / Cant. Reseñas (tri ant)) x 100%*<br>
En el siguiente análisis se muestra cómo se concluyó su reformulación.

In [3]:
df_kpi = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\reseñas_ml.parquet')
df_kpi.head(1)

,review_id,user_id,name,time,rating,text,gmap_id
0,1,1.089912e+20,Song Ro,2021-06-01,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [5]:
# Asegúrate de que la columna 'time' sea del tipo datetime
df_kpi['time'] = pd.to_datetime(df_kpi['time'])

# Extraer el año
df_kpi['año'] = df_kpi['time'].dt.year

# Extraer el trimestre
df_kpi['trimestre'] = df_kpi['time'].dt.quarter

# Extraer el bimestre (1 = enero-febrero, 2 = marzo-abril, etc.)
df_kpi['bimestre'] = ((df_kpi['time'].dt.month - 1) // 2) + 1

# Mostrar el DataFrame resultante con las nuevas columnas
print(df_kpi[['time', 'año', 'trimestre', 'bimestre']].head())


         time   año  trimestre  bimestre
0  2021-06-01  2021          2         3
1  2021-09-02  2021          3         5
2  2020-08-03  2020          3         4
3  2019-07-03  2019          3         4
25 2021-02-09  2021          1         1


Para analizar la evolución trimestral de la cantidad de reseñas, nos focalizaremos en los años 2020 y 2021.
Agruparemos el DataFrame df_kpi por gmap_id, año y trimestre, y luego contaremos la cantidad de reseñas. De esa manera calcularemos la evolución promedio trimestral y verificar si se cumple con el KPI del incremento del 5%.

In [7]:
# Agrupar por gmap_id, año y trimestre, y contar las reseñas
df_reseñas_trimestrales = df_kpi.groupby(['gmap_id', 'año', 'trimestre']).size().reset_index(name='cantidad_reseñas')

# Filtrar para los años 2020 y 2021
df_reseñas_trimestrales = df_reseñas_trimestrales[(df_reseñas_trimestrales['año'] == 2020) | (df_reseñas_trimestrales['año'] == 2021)]

# Calcular la evolución promedio trimestral
df_evolucion = df_reseñas_trimestrales.groupby(['año', 'trimestre']).agg({'cantidad_reseñas': 'mean'}).reset_index()

# Calcular el porcentaje de crecimiento trimestral
df_evolucion['crecimiento'] = df_evolucion['cantidad_reseñas'].pct_change() * 100

# Mostrar el resultado
print(df_evolucion)

# Verificar si cumple con el KPI de un crecimiento del 5%
df_evolucion['cumple_kpi'] = df_evolucion['crecimiento'] >= 5

# Mostrar el resultado con el KPI
df_evolucion[['año', 'trimestre', 'cantidad_reseñas', 'crecimiento', 'cumple_kpi']]

    año  trimestre  cantidad_reseñas  crecimiento
0  2020          1          2.545487          NaN
1  2020          2          2.429138    -4.570780
2  2020          3          2.460580     1.294344
3  2020          4          2.465744     0.209887
4  2021          1          1.921875   -22.056992
5  2021          2          1.883730    -1.984788
6  2021          3          1.946456     3.329895
7  2021          4          1.924060    -1.150601


,año,trimestre,cantidad_reseñas,crecimiento,cumple_kpi
0,2020,1,2.545487,NaN,False
1,2020,2,2.429138,-4.570780,False
2,2020,3,2.460580,1.294344,False
3,2020,4,2.465744,0.209887,False
4,2021,1,1.921875,-22.056992,False
5,2021,2,1.883730,-1.984788,False
6,2021,3,1.946456,3.329895,False
7,2021,4,1.924060,-1.150601,False


El porcentaje del 5% (que asignamos arbitrariamente para controlar) ha sido exagerado. Podemos bajarlo al 2%

In [8]:
# Agrupar por gmap_id, año y trimestre, y contar las reseñas
df_reseñas_trimestrales = df_kpi.groupby(['gmap_id', 'año', 'trimestre']).size().reset_index(name='cantidad_reseñas')

# Filtrar para los años 2020 y 2021
df_reseñas_trimestrales = df_reseñas_trimestrales[(df_reseñas_trimestrales['año'] == 2020) | (df_reseñas_trimestrales['año'] == 2021)]

# Calcular la evolución promedio trimestral
df_evolucion = df_reseñas_trimestrales.groupby(['año', 'trimestre']).agg({'cantidad_reseñas': 'mean'}).reset_index()

# Calcular el porcentaje de crecimiento trimestral
df_evolucion['crecimiento'] = df_evolucion['cantidad_reseñas'].pct_change() * 100

# Mostrar el resultado
print(df_evolucion)

# Verificar si cumple con el KPI de un crecimiento del 2%
df_evolucion['cumple_kpi'] = df_evolucion['crecimiento'] >= 2

# Mostrar el resultado con el KPI
df_evolucion[['año', 'trimestre', 'cantidad_reseñas', 'crecimiento', 'cumple_kpi']]

    año  trimestre  cantidad_reseñas  crecimiento
0  2020          1          2.545487          NaN
1  2020          2          2.429138    -4.570780
2  2020          3          2.460580     1.294344
3  2020          4          2.465744     0.209887
4  2021          1          1.921875   -22.056992
5  2021          2          1.883730    -1.984788
6  2021          3          1.946456     3.329895
7  2021          4          1.924060    -1.150601


,año,trimestre,cantidad_reseñas,crecimiento,cumple_kpi
0,2020,1,2.545487,NaN,False
1,2020,2,2.429138,-4.570780,False
2,2020,3,2.460580,1.294344,False
3,2020,4,2.465744,0.209887,False
4,2021,1,1.921875,-22.056992,False
5,2021,2,1.883730,-1.984788,False
6,2021,3,1.946456,3.329895,True
7,2021,4,1.924060,-1.150601,False


Si bien parece cumplirse poco, debemos recordar que los períodos de la muestra contemplan una situación de pandemia global. Podemos ampliar los registros a unos años atrás.

In [9]:
# Agrupar por gmap_id, año y trimestre, y contar las reseñas
df_reseñas_trimestrales = df_kpi.groupby(['gmap_id', 'año', 'trimestre']).size().reset_index(name='cantidad_reseñas')

# Filtrar para los años 2018, 2019, 2020 y 2021
df_reseñas_trimestrales = df_reseñas_trimestrales[(df_reseñas_trimestrales['año'].isin([2018, 2019, 2020, 2021]))]

# Calcular la evolución promedio trimestral
df_evolucion = df_reseñas_trimestrales.groupby(['año', 'trimestre']).agg({'cantidad_reseñas': 'mean'}).reset_index()

# Calcular el porcentaje de crecimiento trimestral
df_evolucion['crecimiento'] = df_evolucion['cantidad_reseñas'].pct_change() * 100

# Mostrar el resultado
print(df_evolucion)

# Verificar si cumple con el KPI de un crecimiento del 2%
df_evolucion['cumple_kpi'] = df_evolucion['crecimiento'] >= 2

# Mostrar el resultado con el KPI
df_evolucion[['año', 'trimestre', 'cantidad_reseñas', 'crecimiento', 'cumple_kpi']]

     año  trimestre  cantidad_reseñas  crecimiento
0   2018          1          3.719320          NaN
1   2018          2          3.616907    -2.753537
2   2018          3          3.614701    -0.060994
3   2018          4          3.739218     3.444736
4   2019          1          3.674097    -1.741560
5   2019          2          3.724031     1.359073
6   2019          3          3.683477    -1.088989
7   2019          4          3.717449     0.922291
8   2020          1          2.545487   -31.525983
9   2020          2          2.429138    -4.570780
10  2020          3          2.460580     1.294344
11  2020          4          2.465744     0.209887
12  2021          1          1.921875   -22.056992
13  2021          2          1.883730    -1.984788
14  2021          3          1.946456     3.329895
15  2021          4          1.924060    -1.150601


,año,trimestre,cantidad_reseñas,crecimiento,cumple_kpi
0,2018,1,3.719320,NaN,False
1,2018,2,3.616907,-2.753537,False
2,2018,3,3.614701,-0.060994,False
3,2018,4,3.739218,3.444736,True
4,2019,1,3.674097,-1.741560,False
5,2019,2,3.724031,1.359073,False
6,2019,3,3.683477,-1.088989,False
7,2019,4,3.717449,0.922291,False
8,2020,1,2.545487,-31.525983,False
9,2020,2,2.429138,-4.570780,False


El porcentaje parece aceptable, pero no se demuestra de esa forma en su cumplimiento. Lo que haremos es traer otro dataframe para filtrar los condados de interés.

In [10]:
df_locales = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\locales_google.parquet')
df_locales.columns

Index(['gmap_id', 'name', 'address', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'state', 'relative_results',
       'Service options', 'clasificacion', 'address_depurada', 'city',
       'county'],
      dtype='object')

In [11]:
# Filtrar el DataFrame df_locales para obtener los gmap_id de los condados de interés
condados_interes = ['Los Angeles', 'San Francisco', 'Orange', 'San Diego']
df_gmap_ids = df_locales[df_locales['county'].isin(condados_interes)]

# Obtener los gmap_id
gmap_ids_interes = df_gmap_ids['gmap_id'].unique()
print("gmap_ids de los condados seleccionados:", gmap_ids_interes)


gmap_ids de los condados seleccionados: ['0x80c2c778e3b73d33:0xbdc58662a4a97d49'
 '0x80dd2b4c8555edb7:0xfc33d65c4bdbef42'
 '0x80c2baf50d29bf63:0x5bd904b842b9fcc' ...
 '0x80d953599657345f:0x326882d24ffa795e'
 '0x80c290cd5fcce25d:0x296bf3eca368d132'
 '0x80c2956b34fc26c1:0x2f3b0897def449d6']


Ahora recortaremos la muestra, y calcularemos la evolución sobre los promedios de los condados de interés

In [13]:
# Paso 2: Filtrar el DataFrame df_kpi usando los gmap_id extraídos
df_kpi_filtrado = df_kpi[df_kpi['gmap_id'].isin(gmap_ids_interes)]

# Verificar el tamaño del DataFrame filtrado
print("Tamaño del DataFrame filtrado:", df_kpi_filtrado.shape)

Tamaño del DataFrame filtrado: (78879, 10)


In [14]:
# Agrupar por gmap_id, año y trimestre, y contar las reseñas
df_reseñas_trimestrales_filtrado = df_kpi_filtrado.groupby(['gmap_id', 'año', 'trimestre']).size().reset_index(name='cantidad_reseñas')

# Filtrar para los años 2018, 2019, 2020 y 2021
df_reseñas_trimestrales_filtrado = df_reseñas_trimestrales_filtrado[(df_reseñas_trimestrales_filtrado['año'].isin([2018, 2019, 2020, 2021]))]

# Calcular la evolución promedio trimestral
df_evolucion_filtrado = df_reseñas_trimestrales_filtrado.groupby(['año', 'trimestre']).agg({'cantidad_reseñas': 'mean'}).reset_index()

# Calcular el porcentaje de crecimiento trimestral
df_evolucion_filtrado['crecimiento'] = df_evolucion_filtrado['cantidad_reseñas'].pct_change() * 100

# Mostrar el resultado
print(df_evolucion_filtrado)

# Verificar si cumple con el KPI de un crecimiento del 2%
df_evolucion_filtrado['cumple_kpi'] = df_evolucion_filtrado['crecimiento'] >= 2

# Mostrar el resultado con el KPI
df_evolucion_filtrado[['año', 'trimestre', 'cantidad_reseñas', 'crecimiento', 'cumple_kpi']]


     año  trimestre  cantidad_reseñas  crecimiento
0   2018          1          3.617770          NaN
1   2018          2          3.560935    -1.571007
2   2018          3          3.543210    -0.497763
3   2018          4          3.770505     6.414960
4   2019          1          3.546032    -5.953410
5   2019          2          3.543014    -0.085101
6   2019          3          3.563766     0.585726
7   2019          4          3.602820     1.095858
8   2020          1          2.522529   -29.984598
9   2020          2          2.420000    -4.064535
10  2020          3          2.434274     0.589831
11  2020          4          2.439127     0.199380
12  2021          1          1.875483   -23.108428
13  2021          2          1.823529    -2.770164
14  2021          3          1.882540     3.236047
15  2021          4          1.878220    -0.229453


,año,trimestre,cantidad_reseñas,crecimiento,cumple_kpi
0,2018,1,3.617770,NaN,False
1,2018,2,3.560935,-1.571007,False
2,2018,3,3.543210,-0.497763,False
3,2018,4,3.770505,6.414960,True
4,2019,1,3.546032,-5.953410,False
5,2019,2,3.543014,-0.085101,False
6,2019,3,3.563766,0.585726,False
7,2019,4,3.602820,1.095858,False
8,2020,1,2.522529,-29.984598,False
9,2020,2,2.420000,-4.064535,False


In [17]:
# Calcular si cumple con el KPI de un crecimiento del 0.5%
df_evolucion_filtrado['cumple_kpi'] = df_evolucion_filtrado['crecimiento'] >= 0.5

# Mostrar el resultado con el KPI
print("Resultados con el KPI ajustado:")
print(df_evolucion_filtrado[['año', 'trimestre', 'cantidad_reseñas', 'crecimiento', 'cumple_kpi']])


Resultados con el KPI ajustado:
     año  trimestre  cantidad_reseñas  crecimiento  cumple_kpi
0   2018          1          3.617770          NaN       False
1   2018          2          3.560935    -1.571007       False
2   2018          3          3.543210    -0.497763       False
3   2018          4          3.770505     6.414960        True
4   2019          1          3.546032    -5.953410       False
5   2019          2          3.543014    -0.085101       False
6   2019          3          3.563766     0.585726        True
7   2019          4          3.602820     1.095858        True
8   2020          1          2.522529   -29.984598       False
9   2020          2          2.420000    -4.064535       False
10  2020          3          2.434274     0.589831        True
11  2020          4          2.439127     0.199380       False
12  2021          1          1.875483   -23.108428       False
13  2021          2          1.823529    -2.770164       False
14  2021          3    

El promedio muestra que un 0.5% tiene un mejor cumplimiento sobre el promedio. Lo cierto es que no es una referencia muy clara, porque trabajamos con el promedio. Elegiremos un local con suficientes reseñas y haremos la medición.

In [18]:
df_locales[df_locales['county'].isin(condados_interes)]

,gmap_id,name,address,latitude,longitude,category,avg_rating,num_of_reviews,state,relative_results,Service options,clasificacion,address_depurada,city,county
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",34.058092,-118.292130,Korean restaurant,4.4,18,Open ⋅ Closes 6PM,"0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2c...","Takeout, Dine-in, Delivery",Dining Venue,761 S Vermont Ave,Los Angeles,Los Angeles
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",33.916402,-118.010855,Restaurant,4.5,18,Open ⋅ Closes 9:30PM,Sin resultados vinculados,"Outdoor seating, Curbside pickup, No-contact d...",Dining Venue,12740 La Mirada Blvd,La Mirada,Los Angeles
761,0x80c2baf50d29bf63:0x5bd904b842b9fcc,La Potranca,"La Potranca, 12821 Venice Blvd., Los Angeles, ...",34.000181,-118.441249,Restaurant,4.2,13,Closed ⋅ Opens 10AM,"0x80c2bac345536273:0x8b015c3512788465, 0x80c2c...","Dine-in, Delivery",Dining Venue,12821 Venice Blvd.,Los Angeles,Los Angeles
1548,0x80c2d765f8c90a3d:0x16afb75943e7ad50,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",34.079995,-117.988951,"Hamburger restaurant, American restaurant, Bar...",3.7,38,Closed ⋅ Opens 6AM,"0x80c2d765f8dd4ebf:0xb6baf31e3e536ffa, 0x80c2d...","Drive-through, Takeout, Dine-in, Delivery",Dining Venue,13101 Ramona Blvd,Baldwin Park,Los Angeles
2750,0x80dc736926023cb9:0xb3377acd90a46f16,Chin's Szechwan - Carlsbad,"Chin's Szechwan - Carlsbad, 2820 Roosevelt St,...",33.162363,-117.349109,"Chinese restaurant, Asian fusion restaurant, C...",4.8,48,Closed ⋅ Opens 11:30AM Thu,Sin resultados vinculados,"Curbside pickup, No-contact delivery, Delivery...",Dining Venue,2820 Roosevelt St,Carlsbad,San Diego
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634074,0x808f77904da41faf:0x605e90b35947e9ef,Peets Coffee,"Peets Coffee, 10 Domestic Terminals Departures...",37.618022,-122.387469,Coffee shop,3.7,48,Closed ⋅ Opens 4AM,"0x808f7795d960a7c1:0x20c69961cd236a7c, 0x808f7...","Takeout, Delivery",Café & Coffee,10 Domestic Terminals Departures Level,San Francisco,San Francisco
2634268,0x808f7f6f080d376d:0x7a5c85019dddecf1,Daily Driver,"Daily Driver, 2535 3rd St, San Francisco, CA 9...",37.757314,-122.387939,"Bagel shop, Coffee shop",4.3,195,Closed ⋅ Opens 8AM,"0x808f7e149a5fb643:0x9c516a4384f47748, 0x808f7...","Curbside pickup, In-store pick-up, In-store sh...",Café & Coffee,2535 3rd St,San Francisco,San Francisco
2634616,0x80d953599657345f:0x326882d24ffa795e,Rustic Root,"Rustic Root, 535 Fifth Ave, San Diego, CA 9210...",32.710961,-117.159951,"American restaurant, Brunch restaurant, Cockta...",4.4,1705,Closed ⋅ Opens 5PM Wed,"0x80d954a84fd65131:0xe16f70f023c73bec, 0x80d95...","Outdoor seating, Curbside pickup, Delivery, Ta...",Dining Venue,535 Fifth Ave,San Diego,San Diego
2634673,0x80c290cd5fcce25d:0x296bf3eca368d132,Wendy's,"Wendy's, 14645 Roscoe Blvd, Panorama City, CA ...",34.221861,-118.451103,"Fast food restaurant, Hamburger restaurant, Re...",4.0,1671,Closed ⋅ Opens 6:30AM Mon,"0x80c2972cb1fe7ed5:0xb909c53c800cd077, 0x80c29...","Curbside pickup, Delivery, Drive-through, Take...",Dining Venue,14645 Roscoe Blvd,Panorama City,Los Angeles


In [21]:
# gmap_id del restaurante específico
gmap_id_restaurante = '0x80c290cd5fcce25d:0x296bf3eca368d132'

# Filtrar el DataFrame para el restaurante específico
df_kpi_restaurante = df_kpi[df_kpi['gmap_id'] == gmap_id_restaurante]

# Agrupar por año y trimestre y contar las reseñas
df_reseñas_trimestrales_restaurante = df_kpi_restaurante.groupby(['año', 'trimestre']).size().reset_index(name='cantidad_reseñas')

# Calcular el crecimiento porcentual
df_reseñas_trimestrales_restaurante['crecimiento'] = df_reseñas_trimestrales_restaurante['cantidad_reseñas'].pct_change() * 100

# Verificar si cumple con el KPI de un crecimiento del 1.5%
df_reseñas_trimestrales_restaurante['cumple_kpi'] = df_reseñas_trimestrales_restaurante['crecimiento'] >= 2

# Mostrar el resultado
print("Resultados para el restaurante con gmap_id:", gmap_id_restaurante)
df_reseñas_trimestrales_restaurante[['año', 'trimestre', 'cantidad_reseñas', 'crecimiento', 'cumple_kpi']]


Resultados para el restaurante con gmap_id: 0x80c290cd5fcce25d:0x296bf3eca368d132


,año,trimestre,cantidad_reseñas,crecimiento,cumple_kpi
0,2013,2,1,NaN,False
1,2016,2,2,100.000000,True
2,2016,3,2,0.000000,False
3,2016,4,2,0.000000,False
4,2017,1,1,-50.000000,False
5,2017,4,2,100.000000,True
6,2018,1,17,750.000000,True
7,2018,2,16,-5.882353,False
8,2018,3,18,12.500000,True
9,2018,4,16,-11.111111,False


Como conclusión, podemos ver que el problema no es la cantidad de reseñas, sino la irregularidad de las reseñas. Para corregir esto, en lugar de hacer una comparación directa con el trimestre anterior, podemos plantear un objetivo de crecimiento trimestral, que contemple respecto al período inicial.

In [23]:
# Filtrar el DataFrame para el restaurante específico
df_kpi_restaurante = df_kpi[df_kpi['gmap_id'] == gmap_id_restaurante]

# Agrupar por año y trimestre y contar las reseñas
df_reseñas_trimestrales_restaurante = df_kpi_restaurante.groupby(['año', 'trimestre']).size().reset_index(name='cantidad_reseñas')

# Calcular el total acumulado de reseñas
df_reseñas_trimestrales_restaurante['total_acumulado'] = df_reseñas_trimestrales_restaurante['cantidad_reseñas'].cumsum()

# Establecer el objetivo acumulado (1.5% de crecimiento por trimestre)
# El objetivo inicial es 100% en el primer trimestre
df_reseñas_trimestrales_restaurante['objetivo_acumulado'] =  df_reseñas_trimestrales_restaurante['total_acumulado'].iloc[0] * (1.015 ** (df_reseñas_trimestrales_restaurante.index))

# Verificar si se cumple el KPI de crecimiento acumulado
df_reseñas_trimestrales_restaurante['cumple_kpi'] = df_reseñas_trimestrales_restaurante['total_acumulado'] >= df_reseñas_trimestrales_restaurante['objetivo_acumulado']

# Mostrar el resultado
print("Resultados para el restaurante con gmap_id:", gmap_id_restaurante)
df_reseñas_trimestrales_restaurante[['año', 'trimestre', 'cantidad_reseñas', 'total_acumulado', 'objetivo_acumulado', 'cumple_kpi']]


Resultados para el restaurante con gmap_id: 0x80c290cd5fcce25d:0x296bf3eca368d132


,año,trimestre,cantidad_reseñas,total_acumulado,objetivo_acumulado,cumple_kpi
0,2013,2,1,1,1.000000,True
1,2016,2,2,3,1.015000,True
2,2016,3,2,5,1.030225,True
3,2016,4,2,7,1.045678,True
4,2017,1,1,8,1.061364,True
5,2017,4,2,10,1.077284,True
6,2018,1,17,27,1.093443,True
7,2018,2,16,43,1.109845,True
8,2018,3,18,61,1.126493,True
9,2018,4,16,77,1.143390,True


Ahora veamos respecto al promedio de reseñas

In [25]:
# Filtrar el DataFrame de KPI para los gmap_id de las zonas de interés
df_kpi_zonas_interes = df_kpi[df_kpi['gmap_id'].isin(gmap_ids_interes)]

# Agrupar por año y trimestre, y contar las reseñas promedio
df_reseñas_trimestrales_zonas_interes = df_kpi_zonas_interes.groupby(['año', 'trimestre']).size().reset_index(name='cantidad_reseñas')

# Calcular el promedio de reseñas por trimestre
df_reseñas_trimestrales_zonas_interes['promedio_reseñas'] = df_reseñas_trimestrales_zonas_interes['cantidad_reseñas'].mean()

# Calcular el total acumulado de reseñas
df_reseñas_trimestrales_zonas_interes['total_acumulado'] = df_reseñas_trimestrales_zonas_interes['cantidad_reseñas'].cumsum()

# Establecer el objetivo acumulado (1.5% de crecimiento por trimestre)
# El objetivo inicial es 100% en el primer trimestre
df_reseñas_trimestrales_zonas_interes['objetivo_acumulado'] = df_reseñas_trimestrales_zonas_interes['total_acumulado'].iloc[0] * (1.015 ** (df_reseñas_trimestrales_zonas_interes.index))

# Verificar si se cumple el KPI de crecimiento acumulado
df_reseñas_trimestrales_zonas_interes['cumple_kpi'] = df_reseñas_trimestrales_zonas_interes['total_acumulado'] >= df_reseñas_trimestrales_zonas_interes['objetivo_acumulado']

# Mostrar el resultado
print("Resultados para las zonas de interés:")
df_reseñas_trimestrales_zonas_interes[['año', 'trimestre', 'cantidad_reseñas', 'promedio_reseñas', 'total_acumulado', 'objetivo_acumulado', 'cumple_kpi']]


Resultados para las zonas de interés:


,año,trimestre,cantidad_reseñas,promedio_reseñas,total_acumulado,objetivo_acumulado,cumple_kpi
0,2007,2,2,1359.982759,2,2.000000,True
1,2007,3,1,1359.982759,3,2.030000,True
2,2007,4,1,1359.982759,4,2.060450,True
3,2008,2,3,1359.982759,7,2.091357,True
4,2008,3,1,1359.982759,8,2.122727,True
5,2008,4,3,1359.982759,11,2.154568,True
6,2009,1,2,1359.982759,13,2.186887,True
7,2009,2,3,1359.982759,16,2.219690,True
8,2009,3,2,1359.982759,18,2.252985,True
9,2009,4,5,1359.982759,23,2.286780,True


Ahora que sabemos que tenemos un problema de irregularidad, busquemos el porcentaje. Pasemos a un 5%

In [26]:
# Filtrar el DataFrame de KPI para los gmap_id de las zonas de interés
df_kpi_zonas_interes = df_kpi[df_kpi['gmap_id'].isin(gmap_ids_interes)]

# Agrupar por año y trimestre, y contar las reseñas promedio
df_reseñas_trimestrales_zonas_interes = df_kpi_zonas_interes.groupby(['año', 'trimestre']).size().reset_index(name='cantidad_reseñas')

# Calcular el promedio de reseñas por trimestre
df_reseñas_trimestrales_zonas_interes['promedio_reseñas'] = df_reseñas_trimestrales_zonas_interes['cantidad_reseñas'].mean()

# Calcular el total acumulado de reseñas
df_reseñas_trimestrales_zonas_interes['total_acumulado'] = df_reseñas_trimestrales_zonas_interes['cantidad_reseñas'].cumsum()

# Establecer el objetivo acumulado (1.5% de crecimiento por trimestre)
# El objetivo inicial es 100% en el primer trimestre
df_reseñas_trimestrales_zonas_interes['objetivo_acumulado'] = df_reseñas_trimestrales_zonas_interes['total_acumulado'].iloc[0] * (1.05 ** (df_reseñas_trimestrales_zonas_interes.index))

# Verificar si se cumple el KPI de crecimiento acumulado
df_reseñas_trimestrales_zonas_interes['cumple_kpi'] = df_reseñas_trimestrales_zonas_interes['total_acumulado'] >= df_reseñas_trimestrales_zonas_interes['objetivo_acumulado']

# Mostrar el resultado
print("Resultados para las zonas de interés:")
df_reseñas_trimestrales_zonas_interes[['año', 'trimestre', 'cantidad_reseñas', 'promedio_reseñas', 'total_acumulado', 'objetivo_acumulado', 'cumple_kpi']]


Resultados para las zonas de interés:


,año,trimestre,cantidad_reseñas,promedio_reseñas,total_acumulado,objetivo_acumulado,cumple_kpi
0,2007,2,2,1359.982759,2,2.000000,True
1,2007,3,1,1359.982759,3,2.100000,True
2,2007,4,1,1359.982759,4,2.205000,True
3,2008,2,3,1359.982759,7,2.315250,True
4,2008,3,1,1359.982759,8,2.431013,True
5,2008,4,3,1359.982759,11,2.552563,True
6,2009,1,2,1359.982759,13,2.680191,True
7,2009,2,3,1359.982759,16,2.814201,True
8,2009,3,2,1359.982759,18,2.954911,True
9,2009,4,5,1359.982759,23,3.102656,True


De esta forma, hemos reformulado nuestro KPI:<br>

**KPI: Incrementar en un 5% trimestral la cantidad de reseñas del establecimiento gastronómico, tomando como base el total acumulado de reseñas desde el primer registro.**

In [ ]:
df_kpi

In [4]:
df_kpi.columns

Index(['review_id', 'user_id', 'name', 'time', 'rating', 'text', 'gmap_id'], dtype='object')

**KPI 2: Incrementar en un 5% trimestral la oferta de servicios externos.**

**Definición:** Este KPI tiene el objetivo de aumentar el consumo en los servicios que no requieren consumo en el local (por ejemplo, pick up o delivery), que son más habituales en zonas tech.

**Fórmula:**

*KPI = ((Servicios externos (tri actual) − Servicios externos (tri ant)) / Servicios externos (tri ant)) x 100%*

**KPI 3: Incrementar en un 5% cada bimestre la cantidad de valoraciones de 4 y 5 estrellas.**

**Definición:** Este KPI mide la evolución de la percepción positiva de los clientes sobre la calidad del establecimiento, buscando un crecimiento continuo en las valoraciones positivas (4 o más estrellas) cada dos meses.

**Fórmula:**

*KPI = ((Cant. ≥4 (bimestre actual) − Cant. ≥4 (bimestre ant)) / Cant. ≥4 (bimestre ant)) x 100%*